# Fast Calibrated Explanations for Binary Classification
## Explanation Time Analysis

Author: Tuwe Löfström (tuwe.lofstrom@ju.se)  
Copyright 2023 Tuwe Löfström  
License: BSD 3 clause

### 1. Import packages

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd
import numpy as np
from scipy import stats as st

### 2 Import results from the pickled result file

In [3]:
with open('../../results_perturbed_ablation.pkl', 'rb') as f:
    results = pickle.load(f)
data_characteristics = {'colic': 60, 
                        'creditA': 43, 
                        'diabetes': 9, 
                        'german': 28, 
                        'haberman': 4, 
                        'haberman': 4,
                        'heartC': 23,
                        'heartH': 21,
                        'heartS': 14,
                        'hepati': 20,
                        'iono': 34,
                        'je4042': 9,
                        'je4243': 9, 
                        'kc1': 22,
                        'kc2': 22,
                        'kc3': 40,
                        'liver': 7,
                        'pc1req': 9,
                        'pc4': 38,
                        'sonar': 61,
                        'spect': 23,
                        'spectf': 45,
                        'transfusion': 5,
                        'ttt': 28,
                        'vote': 17,
                        'wbc': 10,}

In [4]:
print(results.keys())

dict_keys(['severity', 'noise_type', 'scale_factor', 'test_size', 'pc1req', 'haberman', 'hepati', 'transfusion', 'spect', 'heartS', 'heartH', 'heartC', 'je4243', 'vote', 'kc2', 'wbc', 'kc3', 'creditA', 'diabetes', 'iono', 'liver', 'je4042', 'sonar', 'spectf', 'german', 'ttt', 'colic', 'pc4', 'kc1'])


In [5]:
timer = results['pc1req']['RF']['timer']
pce_init = timer['pce_init']
pce_explain = timer['pce_explain']

# Remove 'severity', 'noise_type', 'scale_factor', and 'test_size' from results and store in a new dict
filtered_results = {k: v for k, v in results.items() if k not in ['severity', 'noise_type', 'scale_factor', 'test_size']}

# Create a DataFrame for pce_init
df_pce = pd.DataFrame.from_dict({(i, j, k, l, m): filtered_results[i][j]['timer']['pce_init'][k][l][m]
                                   for i in filtered_results.keys() 
                                   for j in filtered_results[i].keys() 
                                   for k in filtered_results[i][j]['timer']['pce_init'].keys()
                                   for l in filtered_results[i][j]['timer']['pce_init'][k].keys()
                                   for m in filtered_results[i][j]['timer']['pce_init'][k][l].keys()},
                                  orient='index')
df_pce.reset_index(inplace=True)
df_pce.columns = ['dataset - algorithm - scale_factor - severity - noise_type', 'init_time']

# Create a DataFrame for pce_init
df_pce_explain = pd.DataFrame.from_dict({(i, j, k, l, m): filtered_results[i][j]['timer']['pce_explain'][k][l][m]
                                   for i in filtered_results.keys() 
                                   for j in filtered_results[i].keys() 
                                   for k in filtered_results[i][j]['timer']['pce_explain'].keys()
                                   for l in filtered_results[i][j]['timer']['pce_explain'][k].keys()
                                   for m in filtered_results[i][j]['timer']['pce_explain'][k][l].keys()},
                                  orient='index')
df_pce_explain.reset_index(inplace=True)
df_pce_explain.columns = ['dataset - algorithm - scale_factor - severity - noise_type', 'explain_time']

# Merge the two DataFrames on 'scale_factor - severity - noise_type'
df_pce = pd.merge(df_pce, df_pce_explain, on='dataset - algorithm - scale_factor - severity - noise_type')
print(df_pce)

     dataset - algorithm - scale_factor - severity - noise_type  init_time  \
0                          (pc1req, xGB, 1, 0, uniform)           0.000538   
1                         (pc1req, xGB, 1, 0, gaussian)           0.000577   
2                       (pc1req, xGB, 1, 0.25, uniform)           0.000558   
3                      (pc1req, xGB, 1, 0.25, gaussian)           0.000577   
4                        (pc1req, xGB, 1, 0.5, uniform)           0.000538   
...                                                 ...                ...   
1995                       (kc1, RF, 10, 0.5, gaussian)           0.001327   
1996                       (kc1, RF, 10, 0.75, uniform)           0.001342   
1997                      (kc1, RF, 10, 0.75, gaussian)           0.001316   
1998                          (kc1, RF, 10, 1, uniform)           0.001362   
1999                         (kc1, RF, 10, 1, gaussian)           0.001344   

      explain_time  
0         0.000923  
1         0.000885  


In [6]:
criteria = {'scale_factor': 2, 'severity': 3, 'noise_type': 4}
# BEGIN: Calculate average time per scale_factor
for c, idx in criteria.items():
    df_pce[c] = df_pce['dataset - algorithm - scale_factor - severity - noise_type'].apply(lambda x: x[idx])
    average_time_per_scale_factor = df_pce.groupby(c)[['init_time','explain_time']].mean()
    print(average_time_per_scale_factor)
# END: Calculate average time per scale_factor

              init_time  explain_time
scale_factor                         
1              0.001307      0.001868
3              0.001472      0.001659
5              0.001580      0.001664
10             0.001783      0.001645
          init_time  explain_time
severity                         
0.00       0.001439      0.001652
0.25       0.001564      0.001939
0.50       0.001569      0.001665
0.75       0.001559      0.001656
1.00       0.001546      0.001633
            init_time  explain_time
noise_type                         
gaussian     0.001543      0.001774
uniform      0.001528      0.001644


In [7]:
# Create a DataFrame for pce_init
df_ce = pd.DataFrame.from_dict({(i, j): filtered_results[i][j]['timer']['ce_init']
                                   for i in filtered_results.keys() 
                                   for j in filtered_results[i].keys()},
                                  orient='index')
df_ce.reset_index(inplace=True)
df_ce.columns = ['dataset - algorithm', 'init_time']

# Create a DataFrame for ce_init
df_ce_explain = pd.DataFrame.from_dict({(i, j): filtered_results[i][j]['timer']['ce_explain']
                                   for i in filtered_results.keys() 
                                   for j in filtered_results[i].keys()},
                                  orient='index')
df_ce_explain.reset_index(inplace=True)
df_ce_explain.columns = ['dataset - algorithm', 'explain_time']

# Merge the two DataFrames on 'dataset - algorithm'
df_ce = pd.merge(df_ce, df_ce_explain, on='dataset - algorithm')
# print(df_ce)


In [8]:
# Pivot the DataFrame to get datasets as rows and criteria and algorithm as columns
pivot_df = df_ce.pivot_table(index='dataset - algorithm', 
                              values=['init_time', 'explain_time'], 
                              aggfunc='mean')

# Reset the index to make it easier to work with
pivot_df.reset_index(inplace=True)

# Split the 'dataset - algorithm - scale_factor - severity - noise_type' column into separate columns
pivot_df[['dataset', 'algorithm']] = pd.DataFrame(pivot_df['dataset - algorithm'].tolist(), index=pivot_df.index)

# Drop the original combined column
pivot_df.drop(columns=['dataset - algorithm'], inplace=True)

# Pivot again to get the desired format
final_df = pivot_df.pivot_table(index='dataset', 
                                columns=['algorithm'], 
                                values=['init_time', 'explain_time'], 
                                aggfunc='mean')

final_df['Num Features'] = data_characteristics.values()
# Print the final DataFrame
display(final_df)

final_df.to_csv('ce_init_explain_time.csv')


explain_time           init_time           Num Features
algorithm             RF       xGB        RF       xGB             
dataset                                                            
colic           0.368525  0.359768  0.000067  0.000056           60
creditA         0.594193  0.594193  0.000040  0.000038           43
diabetes        0.034349  0.035503  0.000047  0.000036            9
german          0.210025  1.170071  0.000033  0.000029           28
haberman        0.004335  0.005488  0.000071  0.000056            4
heartC          0.058349  0.063026  0.000079  0.000079           23
heartH          0.044721  0.047814  0.000095  0.000055           21
heartS          0.031540  0.032183  0.000088  0.000059           14
hepati          0.033888  0.037455  0.000231  0.000103           20
iono            0.235946  0.258545  0.000068  0.000080           34
je4042          0.021408  0.024359  0.000087  0.000044            9
je4243          0.028348  0.040301  0.000066  0.000033            9
kc1             0.340137  0.330636  0.000023  0.000027           22
kc2             0.121135  0.123276  0.000054  0.000043           22
kc3             0.474546  0.370638  0.000074  0.000049           40
liver           0.010956  0.012151  0.000071  0.000047            7
pc1req          0.009002  0.010870  0.000192  0.000154            9
pc4             1.336231  1.331604  0.000021  0.000021           38
sonar           0.832718  0.644311  0.000298  0.000077           61
spect           0.031535  0.038002  0.000082  0.000055           23
spectf          0.420692  0.412993  0.000089  0.000060           45
transfusion     0.007759  0.007949  0.000048  0.000040            5
ttt             0.168043  0.189444  0.000033  0.000048           28
vote            0.045135  0.036569  0.000070  0.000039           17
wbc             0.038086  0.038659  0.000052  0.000043           10

In [9]:
mean_df = pivot_df.pivot_table(columns=['algorithm'], 
                                values=['init_time', 'explain_time'], 
                                aggfunc='mean')
display(mean_df)

algorithm,RF,xGB
explain_time,0.220064,0.248632
init_time,0.000083,0.000055


In [10]:
# Pivot the DataFrame to get datasets as rows and criteria and algorithm as columns
from typing import final


pivot_df = df_pce.pivot_table(index='dataset - algorithm - scale_factor - severity - noise_type', 
                              values=['init_time', 'explain_time'], 
                              aggfunc='mean')

# Reset the index to make it easier to work with
pivot_df.reset_index(inplace=True)

# Split the 'dataset - algorithm - scale_factor - severity - noise_type' column into separate columns
pivot_df[['dataset', 'algorithm', 'scale_factor', 'severity', 'noise_type']] = pd.DataFrame(pivot_df['dataset - algorithm - scale_factor - severity - noise_type'].tolist(), index=pivot_df.index)

# Drop the original combined column
pivot_df.drop(columns=['dataset - algorithm - scale_factor - severity - noise_type'], inplace=True)

filtered_df = pivot_df

# Pivot again to get the desired format
final_df = filtered_df.pivot_table(index='dataset', 
                                   columns=['algorithm', 'noise_type', 'scale_factor', 'severity'], 
                                   values=['explain_time', 'init_time', ], 
                                   aggfunc='mean')

# Print the final DataFrame
# display(final_df)
final_df.to_csv('explain_time.csv')

# Filter the DataFrame to include only rows where noise_type is 'uniform'
filtered_df = pivot_df[pivot_df['noise_type'] == 'uniform']

# Pivot again to get the desired format
final_df = filtered_df.pivot_table(index='dataset', 
                                   columns=['noise_type', 'scale_factor', 'severity'], 
                                   values=['explain_time'], 
                                   aggfunc='mean')

# Print the final DataFrame
display(final_df)

# Filter the DataFrame to include only rows where noise_type is 'uniform'
filtered_df = pivot_df[pivot_df['noise_type'] == 'gaussian']

# Pivot again to get the desired format
final_df = filtered_df.pivot_table(index='dataset', 
                                   columns=['noise_type', 'scale_factor', 'severity'], 
                                   values=['explain_time'], 
                                   aggfunc='mean')

# Print the final DataFrame
display(final_df)




explain_time                                                    \
noise_type        uniform                                                     
scale_factor           1                                                 3    
severity             0.00      0.25      0.50      0.75      1.00      0.00   
dataset                                                                       
colic            0.003388  0.003335  0.003392  0.003271  0.003278  0.003407   
creditA          0.001747  0.001830  0.001860  0.001847  0.002018  0.002051   
diabetes         0.000410  0.000687  0.000516  0.000748  0.000532  0.000542   
german           0.001231  0.001284  0.001183  0.004963  0.001363  0.001429   
haberman         0.000420  0.000497  0.000461  0.000494  0.000443  0.000433   
heartC           0.001386  0.001473  0.001470  0.001443  0.001463  0.001575   
heartH           0.001296  0.001364  0.001488  0.001381  0.001404  0.001839   
heartS           0.000935  0.001049  0.001064  0.001030  0.001100  0.001056   
hepati           0.002061  0.002488  0.002605  0.002222  0.002344  0.002594   
iono             0.001767  0.002031  0.002050  0.002057  0.002249  0.002211   
je4042           0.001320  0.000727  0.000817  0.000740  0.000743  0.000862   
je4243           0.000583  0.000625  0.000669  0.000622  0.000663  0.000621   
kc1              0.000937  0.001045  0.001027  0.000996  0.001010  0.001025   
kc2              0.001145  0.001488  0.001408  0.001590  0.001468  0.001603   
kc3              0.002536  0.004108  0.003409  0.003962  0.002910  0.003065   
liver            0.000482  0.000560  0.000576  0.000594  0.000587  0.000579   
pc1req           0.001672  0.001385  0.001617  0.001398  0.001473  0.001629   
pc4              0.001350  0.001577  0.001572  0.001579  0.001544  0.001558   
sonar            0.004325  0.004593  0.004727  0.005115  0.004910  0.005025   
spect            0.001790  0.001785  0.001806  0.001680  0.001864  0.001671   
spectf           0.003197  0.005167  0.003402  0.003188  0.002987  0.003175   
transfusion      0.000332  0.000369  0.000701  0.000442  0.000369  0.000347   
ttt              0.001365  0.001154  0.001141  0.001137  0.001147  0.001174   
vote             0.000957  0.000878  0.000958  0.000947  0.000894  0.001212   
wbc              0.000516  0.000615  0.000689  0.001012  0.000637  0.000704   

                                                                          \
noise_type                                                                 
scale_factor                                                5              
severity          0.25      0.50      0.75      1.00      0.00      0.25   
dataset                                                                    
colic         0.003441  0.003428  0.003437  0.003409  0.003846  0.003417   
creditA       0.002400  0.001927  0.001976  0.001983  0.001986  0.001951   
diabetes      0.000545  0.000532  0.000557  0.000533  0.000535  0.000545   
german        0.001356  0.001397  0.001233  0.001226  0.001246  0.001255   
haberman      0.000669  0.000588  0.000483  0.000554  0.000468  0.000518   
heartC        0.001481  0.001516  0.001584  0.001567  0.001612  0.001657   
heartH        0.001452  0.001939  0.001428  0.001442  0.001434  0.001610   
heartS        0.001026  0.001075  0.001085  0.001032  0.001021  0.001024   
hepati        0.002443  0.002262  0.002376  0.002165  0.002523  0.002375   
iono          0.002219  0.002261  0.002276  0.002277  0.002325  0.002370   
je4042        0.000759  0.000706  0.000835  0.000798  0.001324  0.001011   
je4243        0.000615  0.000698  0.000621  0.000638  0.000625  0.000683   
kc1           0.001017  0.001017  0.001013  0.001010  0.001016  0.001190   
kc2           0.001582  0.001477  0.001499  0.001465  0.001544  0.001706   
kc3           0.003300  0.002871  0.002811  0.003428  0.002932  0.002812   
liver         0.000574  0.000568  0.000626  0.000600  0.000583  0.000598   
pc1req        0.001502  0.001472  0.001452  0.

explain_time                                                    \
noise_type       gaussian                                                     
scale_factor           1                                                 3    
severity             0.00      0.25      0.50      0.75      1.00      0.00   
dataset                                                                       
colic            0.004061  0.003319  0.003369  0.003300  0.003333  0.003807   
creditA          0.001831  0.001968  0.001865  0.001838  0.001855  0.001986   
diabetes         0.000499  0.000498  0.000522  0.000521  0.000534  0.000580   
german           0.001433  0.053577  0.001183  0.002187  0.001435  0.001148   
haberman         0.000440  0.000462  0.000447  0.000482  0.000445  0.000522   
heartC           0.001489  0.001495  0.001552  0.001446  0.001501  0.001617   
heartH           0.001353  0.001353  0.001472  0.001445  0.001388  0.001394   
heartS           0.001144  0.001029  0.001057  0.001110  0.001035  0.001028   
hepati           0.002188  0.002189  0.002292  0.002683  0.002273  0.002487   
iono             0.002026  0.002156  0.002097  0.002031  0.002191  0.002240   
je4042           0.000835  0.000748  0.000750  0.000776  0.000801  0.000890   
je4243           0.000693  0.000669  0.000622  0.000641  0.000641  0.000627   
kc1              0.001056  0.001146  0.001034  0.001026  0.001166  0.001206   
kc2              0.001435  0.001400  0.001387  0.001583  0.001439  0.001849   
kc3              0.003002  0.004819  0.004030  0.002918  0.002868  0.004599   
liver            0.000567  0.000566  0.001091  0.000616  0.000584  0.000541   
pc1req           0.001330  0.001367  0.001616  0.001407  0.002494  0.001674   
pc4              0.001581  0.001726  0.001547  0.001557  0.001535  0.001845   
sonar            0.004607  0.004734  0.009200  0.004718  0.004762  0.005275   
spect            0.002344  0.002179  0.001785  0.001708  0.001691  0.001793   
spectf           0.003533  0.005857  0.003129  0.003260  0.003267  0.003413   
transfusion      0.000373  0.000380  0.000574  0.000423  0.000381  0.000352   
ttt              0.001330  0.001149  0.001138  0.001157  0.001135  0.001171   
vote             0.000874  0.000876  0.000995  0.000904  0.001057  0.000904   
wbc              0.000611  0.000646  0.000635  0.000680  0.000648  0.000656   

                                                                          \
noise_type                                                                 
scale_factor                                                5              
severity          0.25      0.50      0.75      1.00      0.00      0.25   
dataset                                                                    
colic         0.003372  0.003487  0.003416  0.003557  0.003425  0.003455   
creditA       0.002109  0.001927  0.001918  0.002311  0.002298  0.001946   
diabetes      0.000547  0.000545  0.000534  0.000756  0.000547  0.000517   
german        0.001480  0.001568  0.001288  0.001189  0.001449  0.001282   
haberman      0.000521  0.000511  0.000650  0.000582  0.000430  0.000511   
heartC        0.001591  0.001496  0.001525  0.001971  0.001521  0.001564   
heartH        0.001587  0.001412  0.001500  0.001406  0.001480  0.001491   
heartS        0.001063  0.001071  0.001053  0.001034  0.001702  0.001063   
hepati        0.002797  0.002201  0.002755  0.002515  0.002176  0.002182   
iono          0.002367  0.002300  0.002733  0.002245  0.002514  0.002329   
je4042        0.000747  0.000828  0.000736  0.000852  0.000799  0.000788   
je4243        0.000603  0.000704  0.000599  0.000617  0.000658  0.000613   
kc1           0.001014  0.001101  0.001016  0.001032  0.001166  0.001024   
kc2           0.001497  0.001447  0.001491  0.001682  0.001763  0.001431   
kc3           0.002793  0.002819  0.002986  0.002800  0.003247  0.002915   
liver         0.000579  0.000606  0.000643  0.000591  0.000623  0.000579   
pc1req        0.001616  0.001635  0.001471  0.

In [16]:
filtered_df = pivot_df[pivot_df['algorithm'] == 'RF']
mean_df = filtered_df.pivot_table(index='severity',
                                columns=['noise_type', 'scale_factor',], 
                                values=['explain_time'], 
                                aggfunc='mean')
display(0.220064/mean_df)
speedup_df = 0.220064/mean_df
speedup_df.to_csv('speedup.csv')

mean_df = filtered_df.pivot_table(index='severity',
                                columns=['noise_type', 'scale_factor',], 
                                values=['init_time'], 
                                aggfunc='mean')
display(mean_df/0.000080)
speeddown_df = mean_df/0.000080
speeddown_df.to_csv('speeddown_init.csv')


explain_time                                                  \
noise_type       gaussian                                         uniform   
scale_factor           1           3           5           10          1    
severity                                                                    
0.00           113.619292  104.916665  110.459264  112.858832  119.771564   
0.25            96.218450  110.471320  111.653355  118.192274  102.726202   
0.50            93.591764  114.532817  111.554868  113.832988  110.234721   
0.75           114.482248  107.960067  116.273983  112.467876  109.488964   
1.00           116.029977  108.985268  114.338363  113.051127  114.876061   

                                                  
noise_type                                        
scale_factor          3           5           10  
severity                                          
0.00          110.362922  107.222780  114.715634  
0.25          108.996115  109.715748  114.164055  
0.50          116.243967  111.503750  116.657388  
0.75          114.329502  109.499684  117.296338  
1.00          115.738312  114.816334  110.468518

init_time                                              \
noise_type     gaussian                                     uniform   
scale_factor         1          3          5          10         1    
severity                                                              
0.00          20.883428  21.911268  23.242681  26.660574  20.291534   
0.25          24.050920  21.663962  24.133107  26.738044  21.146037   
0.50          21.033788  21.890295  23.752576  27.690979  21.703645   
0.75          20.198989  22.211724  24.399914  27.446001  20.658042   
1.00          19.680086  22.073238  23.280382  27.317819  19.842464   

                                               
noise_type                                     
scale_factor         3          5          10  
severity                                       
0.00          22.377206  23.152164  26.522629  
0.25          23.086961  23.885523  27.419754  
0.50          22.365783  24.803382  26.993242  
0.75          22.675206  24.819407  26.929206  
1.00          22.526348  24.165435  27.727767